## 1.匯入所需套件

In [1]:
# Import some useful packages
import matplotlib.pyplot as plt
import numpy as np

# Layers for FNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Layers for CNN
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D

from tensorflow.keras.optimizers import SGD, Adam

# For data preprocessing
from tensorflow.keras import datasets
from tensorflow.keras.utils import to_categorical

## 2.匯入Functional API所需

In [4]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import concatenate, add

## 3.下載並整理 Fashion MNIST 資料集

In [28]:
# Load Fashine MNIST
(x_train, y_train0), (x_test, y_test0) = datasets.fashion_mnist.load_data()

# Reshape size
x_train = x_train.reshape(-1, 28*28)
x_test = x_test.reshape(-1, 28*28)

# Normalize the range of featurs
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

# One-hot encoding
y_train = to_categorical(y_train0, 10)
y_test = to_categorical(y_test0, 10)

## 4.開始打造Functional API
### 我想做一個 1 * 2 * 4 * 2 * 1 的神經網路。
### 也就是說需要輸入及輸出(x,y),輸入為784維的向量，輸出為10維的向量。
### 中間的結果有h1~h8八個 h1,2,7,8為500維的向量，h3,4,5,6為250維的向量。
### 函數有f1~f9九個，上半f1,3,4,7為sigmoid，下半f2,5,6,8為relu，f9為softmax。

In [29]:
f_1 = Dense(500, activation = 'sigmoid')
f_2 = Dense(500, activation = 'relu')
f_3 = Dense(250, activation = 'sigmoid')
f_4 = Dense(250, activation = 'sigmoid')
f_5 = Dense(250, activation = 'relu')
f_6 = Dense(250, activation = 'relu')
f_7 = Dense(500, activation = 'sigmoid')
f_8 = Dense(500, activation = 'relu')
f_9 = Dense(10, activation = 'softmax')

In [30]:
x = Input(shape=(784,))

### 確認輸入x為784維的向量

In [31]:
print(x)

Tensor("input_3:0", shape=(None, 784), dtype=float32)


In [32]:
h_1 = f_1(x)
h_2 = f_2(x)

In [33]:
h_3 = f_3(h_1)
h_4 = f_4(h_1)
h_5 = f_5(h_2)
h_6 = f_6(h_2)

In [34]:
z_1 = concatenate([h_3, h_4])
h_7 = f_7(z_1)
z_2 = concatenate([h_5, h_6])
h_8 = f_8(z_2)

In [35]:
z_3 = concatenate([h_7, h_8])
y = f_9(z_3)

## 確認z_3為1000維的向量，輸出y為10維的向量

In [36]:
print(z_3)
print(y)

Tensor("concatenate_10/Identity:0", shape=(None, 1000), dtype=float32)
Tensor("dense_26/Identity:0", shape=(None, 10), dtype=float32)


In [37]:
model = Model(x, y)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 784)]        0                                            
__________________________________________________________________________________________________
dense_18 (Dense)                (None, 500)          392500      input_3[0][0]                    
__________________________________________________________________________________________________
dense_19 (Dense)                (None, 500)          392500      input_3[0][0]                    
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 250)          125250      dense_18[0][0]                   
____________________________________________________________________________________________

In [38]:
model.compile(loss='mae', 
              optimizer=Adam(), 
              metrics=['accuracy'])

In [39]:
model.fit(x_train, y_train, 
          batch_size=200, 
          epochs=8,
         validation_data = (x_test, y_test)
         )

Train on 60000 samples, validate on 10000 samples
Epoch 1/8
60000/60000 [==============================] - 4s 63us/sample - loss: 0.0595 - accuracy: 0.7077 - val_loss: 0.0539 - val_accuracy: 0.7304
Epoch 2/8
60000/60000 [==============================] - 2s 31us/sample - loss: 0.0530 - accuracy: 0.7359 - val_loss: 0.0522 - val_accuracy: 0.7397
Epoch 3/8
60000/60000 [==============================] - 2s 31us/sample - loss: 0.0523 - accuracy: 0.7391 - val_loss: 0.0552 - val_accuracy: 0.7246
Epoch 4/8
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0519 - accuracy: 0.7406 - val_loss: 0.0551 - val_accuracy: 0.7244
Epoch 5/8
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0514 - accuracy: 0.7429 - val_loss: 0.0545 - val_accuracy: 0.7274
Epoch 6/8
60000/60000 [==============================] - 2s 32us/sample - loss: 0.0522 - accuracy: 0.7389 - val_loss: 0.0539 - val_accuracy: 0.7303
Epoch 7/8
60000/60000 [==============================] - 2s 31

In [40]:
model.save_weights('MNIST_Model.h5')

In [41]:
score = model.evaluate(x_train, y_train, batch_size=10000)
print(f'Loss: {score[0]}')
print(f'準確率: {score[1]*100}')

60000/60000 [==============================] - 0s 8us/sample - loss: 0.0536 - accuracy: 0.7320
Loss: 0.05363441879550616
準確率: 73.19999933242798
